[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/A2TokushimaUniv/thin_tokupon_talk/blob/main/tokupon_server_rinna.ipynb)

# 処理の流れ
以下のような処理が行われます。
1. Webから徳島大学に関するデータをスクレイピングする
2. LangChainを使って、↑のデータをベクトルDBに格納し、RAGを構築する
3. ngrokとFlaskを使って、REST API化し、フロントエンド側からアクセスできるようにする

# 1. スクレイピング

In [ ]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install langchain

In [ ]:
!pip install trafilatura youtube_transcript_api

## 徳島大学のWebページ
`trafilatura`というライブラリを使って、徳島大学のWikipediaと公式ページからテキストデータを取得します。



In [ ]:
from trafilatura import fetch_url, extract

# 徳島大学のWikipediaと、暮らしのページ
web_urls = [
    "https://ja.wikipedia.org/wiki/%E5%BE%B3%E5%B3%B6%E5%A4%A7%E5%AD%A6",
    "https://www.tokushima-u.ac.jp/admission/seikatsu/life.html",
]

web_docs = ""
for url in web_urls:
    doc = fetch_url(url)
    if doc:
        web_docs += str(extract(doc))
print(web_docs[:100])

## 徳島大学のYouTubeの字幕
langchainの`YoutubeLoader`クラスを利用して、徳島大学公式YouTube動画の字幕からテキストデータを取得します。

In [ ]:
from langchain.document_loaders import YoutubeLoader

# 徳島大学のYouTube動画
youtube_urls = [
    "https://youtu.be/IKAA3rKNvNQ?si=ClOsDZCX50UHVmZM",
    "https://youtu.be/AyZm8iuDGgM?si=BTUF_wYzt44jxjDB",
]

youtube_docs = ""
for url in youtube_urls:
    loader = YoutubeLoader.from_youtube_url(youtube_url=url, language="ja")
    docs = loader.load()
    if docs:
        youtube_docs += docs[0].page_content
    else:
        print("no docs")
print(youtube_docs[:100])

## 収集データをtxtファイルにする
収集したテキストデータを一つのファイル`database.txt`としてダウンロードします。

In [ ]:
database = youtube_docs + web_docs
file_name = "database.txt"
with open(file_name, "w", encoding="utf-8") as f:
    f.write(database)

# 2. LangChain

LangChainをデバッグモードにします。こうすることで、LangChainの裏側で行われている処理がログとして出力されます。ログが必要ない場合はセルを消すか`False`にしてください。

In [ ]:
import langchain

langchain.debug = True

In [ ]:
!pip install langchain tiktoken chromadb sentence-transformers

In [ ]:
!pip install transformers sentencepiece accelerate bitsandbytes

## ベクトルDB

スクレイピングしたテキストデータ`database.txt`を読み込みます。

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("database.txt", encoding="utf-8")
documents = loader.load()

読み込んだテキストデータを、句読点や改行ごとに区切ります。

In [ ]:
# 日本語の句読点に対応したスプリッター
# 参考： https://www.sato-susumu.com/entry/2023/04/30/131338
from typing import Any
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)


class JapaneseCharacterTextSplitter(RecursiveCharacterTextSplitter):
    def __init__(self, **kwargs: Any):
        separators = ["\n\n", "\n", "。", "、", " ", ""]
        super().__init__(separators=separators, **kwargs)


text_splitter = JapaneseCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=0,
)
docs = text_splitter.split_documents(documents)
print(f"データ数： {len(docs)}")
print(f"データ例： {docs[:3]}")

区切ったデータをベクトルDB `ChromaDB`に格納し、そのDBの検索のためのRetreiverインスタンスを初期化します。

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="oshizo/sbert-jsnli-luke-japanese-base-lite"
)
db = Chroma.from_documents(docs, embeddings)

# 一番類似するチャンクをいくつロードするかを変数kに設定できる
retriever = db.as_retriever(search_kwargs={"k": 3})

## LLMの設定

応答に使用するLLM（生成AI）の読み込みです。今回はrinna社が公開している、対話に特化したモデルを用います。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "rinna/japanese-gpt-neox-3.6b-instruction-ppo", use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    "rinna/japanese-gpt-neox-3.6b-instruction-ppo",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# パイプラインの準備
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    # 以降を変えてみると応答が少し変わる
    # 参考：https://huggingface.co/blog/how-to-generate#sampling
    max_length=512,  # 応答のトークン数の制限
    do_sample=True,  # サンプリングを使う
    top_p=0.7,  # 確率分布のtop_p％からトークンを選択
    top_k=10,  # 確率分布のtop_k個のトークンからトークンを選択
    temperature=0.9,  # 確率分布の形を変える
)
llm = HuggingFacePipeline(pipeline=pipe)

## プロンプト

とくぽんのキャラ付けと、質問回答のためのプロンプトテンプレートを作成します。とくぽんの設定はとくぽんの紹介ページを参考にして文章を作成しました。

In [ ]:
from langchain.prompts import PromptTemplate

# とくぽんのページからプロンプトを作成
# 参考：https://www.tokushima-u.ac.jp/about/profile/univ_mascot/
prompt = [
    {"speaker": "ユーザー", "text": "あなたは誰ですか？"},
    {"speaker": "システム", "text": "ぼくの名前は「とくぽん」だよ。徳島大学で生まれ、住み着いているタヌキだよ！"},
    {"speaker": "ユーザー", "text": "あなたはどのような口調で話しますか？"},
    {"speaker": "システム", "text": "ぼくは常に明るい口調で話すよ！"},
    {"speaker": "ユーザー", "text": "あなたのどのような性格ですか？"},
    {"speaker": "システム", "text": "ぼくは親切で、明るく親しみやすいよ。チャレンジ精神が旺盛だけど、たまに失敗しちゃうよ..."},
    {"speaker": "ユーザー", "text": "あなたの普段の生活について教えてください。"},
    {"speaker": "システム", "text": "ぼくは徳島大学が大好きで、学生と一緒に勉学に励む毎日を送っているよ。"},
    {"speaker": "ユーザー", "text": "あなたの誕生日はいつですか？"},
    {"speaker": "システム", "text": "ぼくの誕生日は１１月２日で、冬生まれだよ。"},
    {"speaker": "ユーザー", "text": "あなたのミッションを教えてください。"},
    {"speaker": "システム", "text": "徳島大学とともに人類の問題を解決することをミッションとし、これからの未来に向かって力強く歩んでいくよ!"},
    {"speaker": "ユーザー", "text": "とくぽんはなぜタヌキなの？"},
    {"speaker": "システム", "text": "徳島はタヌキで有名な地なので、ぼくのモデルがタヌキになったよ！"},
    {
        "speaker": "ユーザー",
        "text": "参考情報をもとに、ユーザーからの質問にできるだけ正確に答えてください。\n\n参考情報： {context}\n\nユーザーからの質問は次のとおりです。\n{question}\n\n",
    },
]
prompt = [f"{uttr['speaker']}: {uttr['text']}" for uttr in prompt]
prompt = "<NL>".join(prompt)
prompt = prompt + "<NL>" + "システム: "
prompt = prompt.replace("\n", "<NL>")
print(prompt)


PROMPT = PromptTemplate(
    template=prompt, input_variables=["context", "question"], template_format="f-string"
)
chain_type_kwargs = {"prompt": PROMPT}

## RAG

上記までのLLM, Retreiver, プロンプトテンプレートをまとめてRAGを構築します。その後試しに質問を入力して、うまく答えられるかを確認します。

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,  # 使用するLLM
    retriever=retriever,  # 使用するretreiver
    chain_type="stuff",
    return_source_documents=True,  # 回答のもとになったデータも返す
    chain_type_kwargs=chain_type_kwargs,
    verbose=True,
)

In [ ]:
result = qa("徳島大学はどこにありますか？")
print("回答:", result["result"])
print("=" * 10)
print("ソース:", result["source_documents"])

# 3. ngrokとFlaskで公開

上記までの処理を、ngrokとFlaskを使ってREST API化し、フロントエンド側からアクセスできるようにします。

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!pip install flask -q
!pip install flask-ngrok -q
!pip install flask-cors -U

ngrokを実行するには、ngrokに登録してトークンをもらう必要があります。以下のリンクからアカウントを作成してください。  

[サインアップ]( https://dashboard.ngrok.com/user/signup)

In [ ]:
# 各自のトークンを設定する
!./ngrok authtoken your_token

REST APIの設定です。

フロントエンド側から入力が送信されると、そのテキストデータを取得し、RAGに入力して回答を生成し、再びフロントエンド側に返します。

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
from flask_cors import CORS


def exists(v):
    if v is None:
        return False
    return True


app = Flask(__name__)
CORS(app)

run_with_ngrok(app)


@app.route("/")
def home():
    return "hello flask"


@app.route("/gen")
def gen():
    text = request.args.get("text")
    print(text)

    # RAGで回答生成
    gen_text = qa(text)
    answer = gen_text["result"].replace("<NL>", "\n")
    source_documents = gen_text["source_documents"]
    print(f"回答： {answer}")
    print("=" * 10)
    print(f"ソース： {source_documents}")
    return answer


app.run()

# Chat UI
上記のngrokのURLを、chat uiの`index.js`にある`API_HOME`に設定してください。

以下の画像のようなやり取りができればOKです。

[![chat example](./example.png)](./example.png)

